## Zillow Clustering Project
By: Scott Schmidl, Data Scientist
01//2022

### Goal


### Description


### Initial Questions
<p>1)</p>
<p>2)</p>
<p>3)</p>
<p>4)</p>

### Data Dictionary
## TO DO
<table>
<thead><tr>
<th>Target</th>
<th>Meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td>log_error</td>
<td>log(Zestimate) – log(sale price)</td>
</tr>
</tbody>
</table>
<p></p>
<table>
<thead><tr>
<th>Variable</th>
<th>Meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td>parcel_id</td>
<td>unique identifier for parcels (lots)</td>
</tr>
<tr>
<td>bathroom_count</td>
<td>number of bathrooms in home including fractional bathrooms</td>
</tr>
<tr>
<td>bedroom_count</td>
<td>number of bedrooms in home</td>
</tr>
<tr>
<td>home_size_square_feet</td>
<td>calculated total finished living area of the home</td>
</tr>
<tr>
<td>fips</td>
<td>Federal Information Processing Standard code -  see https://en.wikipedia.org/wiki/FIPS_county_code for more details</td>
</tr>
<tr>
<td>latitude</td>
<td>latitude of the middle of the parcel multiplied by 10e6</td>
</tr>
<tr>
<td>longitude</td>
<td>longitude of the middle of the parcel multiplied by 10e6</td>
</tr>
<tr>
<td>lot_size_square_feet</td>
<td>area of the lot in square feet</td>
</tr>
<tr>
<td>raw_census_tract_and_block</td>
<td>census tract and block ID combined - also contains blockgroup assignment by extension</td>
</tr>
<tr>
<td>region_id_city</td>
<td>city in which the property is located (if any)</td>
</tr>
<tr>
<td>region_id_county</td>
<td>county in which the property is located</td>
</tr>
<tr>
<td>region_id_zip</td>
<td>zip code in which the property is located</td>
</tr>
<tr>
<td>room_count</td>
<td>total number of rooms in the principal residence</td>
</tr>
<tr>
<td>year_built</td>
<td>the Year the principal residence was built</td>
</tr>
<tr>
<td>structure_tax_value_price</td>
<td>the assessed value of the built structure on the parcel</td>
</tr>
<tr>
<td>home_price</td>
<td>the total tax assessed value of the parcel</td>
</tr>
<tr>
<td>assessment_year</td>
<td>the year of the property tax assessment</td>
</tr>
<tr>
<td>land_tax_value_price</td>
<td>the assessed value of the land area of the parcel</td>
</tr>
<tr>
<td>tax_amount</td>
<td>the total property tax assessed for that assessment year</td>
</tr>
<tr>
<td>census_tract_and_block</td>
<td>census tract and block ID combined - also contains blockgroup assignment by extension</td>
</tr>
<tr>
<td>transaction_date</td>
<td>date home was purchased</td>
</tr>
<tr>
<td>county</td>
<td>county name of where the home resides</td>
</tr>
</tbody>
</table>

### Acquiring Zillow

### Preparing Zillow

### The Issue

#### Take Away
- 

### Exploratory Data Analysis
- Signifcance Level for each statistical test will be 0.05.

#### Question 1


- <p>H_o:</p>
-  <p>H_a:</p>

#### Take Away
-

#### Question 2


- <p>H_o:</p>
-  <p>H_a:</p>

#### Take Away
- 

#### Question 3


- <p>H_o:</p>
-  <p>H_a:</p>

#### Take Away
- 

#### Question 4


- <p>H_o:</p>
-  <p>H_a:</p>

#### Take Away
- 

#### Key Take Aways
-

### Clustering Zillow

### Baseline
-

### Models
- 

### Conclusion
-

### Recommendations and Next Steps
-